# This notebook provides ways to reformat amplicon read counts and combine with other data sets. Functionalities include converting read counts to relative abundance ('frequencies'), grouping ASVs by taxonomic level, and merging different read count tables.

#### Import all required modules

In [1]:
import pandas as pd
import os as os
import glob as glob
import numpy as np
import re

#### Navigate to your working directory

In [2]:
os.chdir("/Users/nastassia.patin/GitHub/MarineDNA/Data")

### Function to convert counts to relative abundances ("frequencies")

In [102]:
def convert_counts_to_freqs(df):
    df.loc['total']= df.sum()
    df.iloc[:,:] = df.iloc[:,:] / df.iloc[-1,:] # this assumes all columns are samples with counts; can select subset of columns if needed
    df_freq = df[:-1]
    return(df_freq)

## 01. Summarize read counts by taxonomic level (e.g., family)

#### Import the sheets you want to merge

In [17]:
reads = pd.read_csv("RREAS2018_COI_otu_filtered.csv") 
taxa = pd.read_csv("RREAS2018_COI_taxa_filtered.csv")

In [97]:
# ASV read counts
reads = reads.rename(columns={"Unnamed: 0": "ASV"})
reads.head(3)

,ASV,Lasker18Sc122_3_D0052,Lasker18Sc122_5_D0053,Lasker18Sc122_17_D0057,Lasker18Sc122_19_D0058,Lasker18Sc123_3_D0063,Lasker18Sc123_5_D0064,Lasker18Sc123_17_D0068,Lasker18Sc123_19_D0069,Lasker18Sc124_3_D0073,...,Lasker18Sc153_17_D0255,Lasker18Sc153_19_D0256,Lasker18Sc154_3_D0260,Lasker18Sc154_5_D0261,Lasker18Sc154_17_D0265,Lasker18Sc154_19_D0266,Lasker18Sc158_3_D0270,Lasker18Sc158_5_D0271,Lasker18Sc158_17_D0275,Lasker18Sc158_19_D0276
0,ASV_1,3647,6747,10464,9959,274,665,1168,673,346,...,1,13,613,179,2162,1733,743,559,593,605
1,ASV_2,1032,1461,753,886,984,1770,2764,1387,276,...,5,102,0,0,714,503,217,146,290,301
2,ASV_3,2347,3,2206,0,1630,2233,1,3117,5,...,4,1,5780,2423,1,1,22,4709,1,1


In [98]:
# ASV taxonomic lineages
taxa = taxa.rename(columns={"Unnamed: 0": "ASV"})
taxa.head(3)

,ASV,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,ASV_1,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned,unassigned
1,ASV_2,Eukaryota,Haptista,Haptophyta,Isochrysidales,Noelaerhabdaceae,Emiliania,Emiliania huxleyi
2,ASV_3,Eukaryota,Arthropoda,Hexanauplia,Poecilostomatoida,Oncaeidae,g_,s_


### Choose taxonomic group of interest to assign to each ASV

In [99]:
fam = taxa[['ASV', 'Family']]
fam.head(3)

,ASV,Family
0,ASV_1,unassigned
1,ASV_2,Noelaerhabdaceae
2,ASV_3,Oncaeidae


### Merge ASV count sheet with family-level assignments

In [100]:
df = pd.merge(reads, fam, on='ASV').set_index('ASV')
df.head(3)

,Lasker18Sc122_3_D0052,Lasker18Sc122_5_D0053,Lasker18Sc122_17_D0057,Lasker18Sc122_19_D0058,Lasker18Sc123_3_D0063,Lasker18Sc123_5_D0064,Lasker18Sc123_17_D0068,Lasker18Sc123_19_D0069,Lasker18Sc124_3_D0073,Lasker18Sc124_5_D0074,...,Lasker18Sc153_19_D0256,Lasker18Sc154_3_D0260,Lasker18Sc154_5_D0261,Lasker18Sc154_17_D0265,Lasker18Sc154_19_D0266,Lasker18Sc158_3_D0270,Lasker18Sc158_5_D0271,Lasker18Sc158_17_D0275,Lasker18Sc158_19_D0276,Family
ASV,,,,,,,,,,,,,,,,,,,,,
ASV_1,3647,6747,10464,9959,274,665,1168,673,346,828,...,13,613,179,2162,1733,743,559,593,605,unassigned
ASV_2,1032,1461,753,886,984,1770,2764,1387,276,885,...,102,0,0,714,503,217,146,290,301,Noelaerhabdaceae
ASV_3,2347,3,2206,0,1630,2233,1,3117,5,8,...,1,5780,2423,1,1,22,4709,1,1,Oncaeidae


In [101]:
df = df.groupby('Family').sum()
df

,Lasker18Sc122_3_D0052,Lasker18Sc122_5_D0053,Lasker18Sc122_17_D0057,Lasker18Sc122_19_D0058,Lasker18Sc123_3_D0063,Lasker18Sc123_5_D0064,Lasker18Sc123_17_D0068,Lasker18Sc123_19_D0069,Lasker18Sc124_3_D0073,Lasker18Sc124_5_D0074,...,Lasker18Sc153_17_D0255,Lasker18Sc153_19_D0256,Lasker18Sc154_3_D0260,Lasker18Sc154_5_D0261,Lasker18Sc154_17_D0265,Lasker18Sc154_19_D0266,Lasker18Sc158_3_D0270,Lasker18Sc158_5_D0271,Lasker18Sc158_17_D0275,Lasker18Sc158_19_D0276
Family,,,,,,,,,,,,,,,,,,,,,
Acanthamoebidae,196,1,7,2,9,55,2,0,20,17,...,0,0,218,42,3,0,44,3,3,0
Acartiidae,0,1,4,0,0,3,277,826,0,0,...,13110,4009,2,0,2079,7399,1,44,2328,1972
Achatinellidae,0,0,0,0,0,0,0,0,0,0,...,26,25,0,3,76,99,0,3,26,10
Acinetosporaceae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aeginidae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitrinidae,3,6,0,0,0,0,0,0,0,0,...,0,0,12,0,0,0,0,4,0,0
Zancleidae,0,0,0,2,0,0,0,0,0,0,...,0,2,3,0,1,2,1,1,0,0
no_hit,8976,21336,16713,14474,4504,12589,135,74,6399,14060,...,4461,6266,8744,8789,5309,5492,6936,3421,6310,3509


In [29]:
df.to_csv("RREAS2018_COI_families.csv")

### Convert family read counts to relative abundancies ("frequencies")

In [103]:
df_family_freqs = convert_counts_to_freqs(df)
df_family_freqs.head(3)

,Lasker18Sc122_3_D0052,Lasker18Sc122_5_D0053,Lasker18Sc122_17_D0057,Lasker18Sc122_19_D0058,Lasker18Sc123_3_D0063,Lasker18Sc123_5_D0064,Lasker18Sc123_17_D0068,Lasker18Sc123_19_D0069,Lasker18Sc124_3_D0073,Lasker18Sc124_5_D0074,...,Lasker18Sc153_17_D0255,Lasker18Sc153_19_D0256,Lasker18Sc154_3_D0260,Lasker18Sc154_5_D0261,Lasker18Sc154_17_D0265,Lasker18Sc154_19_D0266,Lasker18Sc158_3_D0270,Lasker18Sc158_5_D0271,Lasker18Sc158_17_D0275,Lasker18Sc158_19_D0276
Family,,,,,,,,,,,,,,,,,,,,,
Acanthamoebidae,0.005754,0.000015,0.000092,0.000029,0.000252,0.000803,0.000030,0.000000,0.000417,0.000252,...,0.000000,0.000000,0.006629,0.001380,0.000043,0.000000,0.001188,0.000119,0.000048,0.000000
Acartiidae,0.000000,0.000015,0.000052,0.000000,0.000000,0.000044,0.004130,0.015868,0.000000,0.000000,...,0.235089,0.086282,0.000061,0.000000,0.029871,0.112864,0.000027,0.001739,0.037400,0.037606
Achatinellidae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000466,0.000538,0.000000,0.000099,0.001092,0.001510,0.000000,0.000119,0.000418,0.000191
Acinetosporaceae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Aeginidae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitrinidae,0.000088,0.000093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000365,0.000000,0.000000,0.000000,0.000000,0.000158,0.000000,0.000000
Zancleidae,0.000000,0.000000,0.000000,0.000029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000043,0.000091,0.000000,0.000014,0.000031,0.000027,0.000040,0.000000,0.000000
no_hit,0.263512,0.329590,0.218989,0.209750,0.126208,0.183851,0.002013,0.001422,0.133340,0.208639,...,0.079995,0.134857,0.265872,0.288808,0.076279,0.083774,0.187303,0.135169,0.101372,0.066916


In [104]:
df_family_freqs.to_csv("RREAS2018_COI_families_freq.csv")

### Select one family of interest

In [105]:
df_freq = df_family_freqs.T
df_freq = df_freq[['Noelaerhabdaceae']]
df_freq = df_freq.rename_axis('Sample')
df_freq

Family,Noelaerhabdaceae
Sample,
Lasker18Sc122_3_D0052,0.031148
Lasker18Sc122_5_D0053,0.023434
Lasker18Sc122_17_D0057,0.010784
Lasker18Sc122_19_D0058,0.014332
Lasker18Sc123_3_D0063,0.028274
...,...
Lasker18Sc154_19_D0266,0.010007
Lasker18Sc158_3_D0270,0.006211
Lasker18Sc158_5_D0271,0.006638


In [106]:
df_freq.to_csv("RREAS2018_COI_Noelaerhab_freq.csv")

## 02. Merge selected taxon/taxa frequences with a 16S table

In [107]:
df_16S = pd.read_csv("RREAS2018_16S_table_counts.tsv", sep='\t')
df_16S = df_16S.set_index("ASV ID")
df_16S

,Flyer18Fc03_10_1,Flyer18Fc03_3_1,Flyer18Fc09_10_1,Flyer18Fc09_4_1,Flyer18Fc13_10_1,Flyer18Fc13_3_1,Lasker18Sc122_17,Lasker18Sc122_19,Lasker18Sc122_3,Lasker18Sc122_5,...,Lasker18Sc153_3,Lasker18Sc153_5,Lasker18Sc154_17,Lasker18Sc154_19,Lasker18Sc154_3,Lasker18Sc154_5,Lasker18Sc158_17,Lasker18Sc158_19,Lasker18Sc158_3,Lasker18Sc158_5
ASV ID,,,,,,,,,,,,,,,,,,,,,
a404bf4d32831f2133794e5fe5d0a73e,117,6319,4111,80,3916,151,514,452,5608,2089,...,3377,5228,232,283,5670,5001,78,49,5965,6185
f35c0a576bb1ab3577191cb12352c249,419,58,45,322,22,155,176,149,234,62,...,615,811,5053,7187,166,112,7859,7144,480,992
588253f8c914de60cbb9577946854a39,1975,1843,3545,2913,2398,5836,16700,14422,2638,1022,...,1156,1871,517,798,722,504,229,156,2527,2998
d883e3b29658c861ac524bbfc2cef34d,124,39,31,80,0,0,0,0,63,0,...,322,281,10969,13535,142,144,6950,6198,265,728
3468430277ce2317dec6f413dcfaa03e,1440,31,115,1258,29,434,480,367,97,41,...,266,345,2336,3027,81,49,2969,2279,325,507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5dc6c2efdb748cb3508b4e8592239b83,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6f477eb9c90036177a1a870f0d4d5cc8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
fd5567754d768cc191c36f6aaa99c51d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Convert 16S counts to relative abundances ("frequencies")

In [108]:
df_16S_freqs = convert_counts_to_freqs(df_16S)
df_16S_freqs.head(3)

,Flyer18Fc03_10_1,Flyer18Fc03_3_1,Flyer18Fc09_10_1,Flyer18Fc09_4_1,Flyer18Fc13_10_1,Flyer18Fc13_3_1,Lasker18Sc122_17,Lasker18Sc122_19,Lasker18Sc122_3,Lasker18Sc122_5,...,Lasker18Sc153_3,Lasker18Sc153_5,Lasker18Sc154_17,Lasker18Sc154_19,Lasker18Sc154_3,Lasker18Sc154_5,Lasker18Sc158_17,Lasker18Sc158_19,Lasker18Sc158_3,Lasker18Sc158_5
ASV ID,,,,,,,,,,,,,,,,,,,,,
a404bf4d32831f2133794e5fe5d0a73e,0.001889,0.068789,0.05847,0.001555,0.063234,0.004260,0.006743,0.006891,0.051873,0.051078,...,0.072385,0.072431,0.003481,0.003042,0.098849,0.111341,0.001213,0.000904,0.105980,0.087640
f35c0a576bb1ab3577191cb12352c249,0.006766,0.000631,0.00064,0.006260,0.000355,0.004373,0.002309,0.002272,0.002164,0.001516,...,0.013182,0.011236,0.075819,0.077244,0.002894,0.002494,0.122211,0.131803,0.008528,0.014056
588253f8c914de60cbb9577946854a39,0.031891,0.020063,0.05042,0.056636,0.038722,0.164640,0.219094,0.219868,0.024401,0.024989,...,0.024779,0.025922,0.007757,0.008577,0.012587,0.011221,0.003561,0.002878,0.044897,0.042481


### Reformat table count of interest for merging

In [109]:
df_noel = df_freq.T.reset_index()
df_noel = df_noel.rename(columns={"Family" : "ASV ID"})
df_noel = df_noel.set_index("ASV ID")
df_noel

Sample,Lasker18Sc122_3_D0052,Lasker18Sc122_5_D0053,Lasker18Sc122_17_D0057,Lasker18Sc122_19_D0058,Lasker18Sc123_3_D0063,Lasker18Sc123_5_D0064,Lasker18Sc123_17_D0068,Lasker18Sc123_19_D0069,Lasker18Sc124_3_D0073,Lasker18Sc124_5_D0074,...,Lasker18Sc153_17_D0255,Lasker18Sc153_19_D0256,Lasker18Sc154_3_D0260,Lasker18Sc154_5_D0261,Lasker18Sc154_17_D0265,Lasker18Sc154_19_D0266,Lasker18Sc158_3_D0270,Lasker18Sc158_5_D0271,Lasker18Sc158_17_D0275,Lasker18Sc158_19_D0276
ASV ID,,,,,,,,,,,,,,,,,,,,,
Noelaerhabdaceae,0.031148,0.023434,0.010784,0.014332,0.028274,0.026579,0.041543,0.026722,0.006376,0.013429,...,0.000753,0.002798,0.000213,0.000493,0.012213,0.010007,0.006211,0.006638,0.005269,0.006407


#### Get rid of DNA ID in sample names

In [110]:
df_noel = df_noel.rename(columns=lambda x: re.sub('_D.*$', '', x))
df_noel

Sample,Lasker18Sc122_3,Lasker18Sc122_5,Lasker18Sc122_17,Lasker18Sc122_19,Lasker18Sc123_3,Lasker18Sc123_5,Lasker18Sc123_17,Lasker18Sc123_19,Lasker18Sc124_3,Lasker18Sc124_5,...,Lasker18Sc153_17,Lasker18Sc153_19,Lasker18Sc154_3,Lasker18Sc154_5,Lasker18Sc154_17,Lasker18Sc154_19,Lasker18Sc158_3,Lasker18Sc158_5,Lasker18Sc158_17,Lasker18Sc158_19
ASV ID,,,,,,,,,,,,,,,,,,,,,
Noelaerhabdaceae,0.031148,0.023434,0.010784,0.014332,0.028274,0.026579,0.041543,0.026722,0.006376,0.013429,...,0.000753,0.002798,0.000213,0.000493,0.012213,0.010007,0.006211,0.006638,0.005269,0.006407


In [111]:
df = pd.concat([df_16S_freqs, df_noel], join='inner', sort=False)
df = df.T
df

ASV ID,a404bf4d32831f2133794e5fe5d0a73e,f35c0a576bb1ab3577191cb12352c249,588253f8c914de60cbb9577946854a39,d883e3b29658c861ac524bbfc2cef34d,3468430277ce2317dec6f413dcfaa03e,625cabfbd66e4f32de52ec86cb441744,af4ec853b8f3af897984b19fd4b71d91,d42ff5ae074d8ae7b783eac352a53d9b,6b9dd64d080d53d989a8407d9806429e,76dfe03cf1770d4ae5c703515096f2e9,...,80c4735759e2048a7cc2f56f19a1f304,1349330a18891c59f576209fe532626b,ebe4de7f14399b72623a976ee09b8e77,78567cbcdf70b7f2c368a1870633f111,5dc6c2efdb748cb3508b4e8592239b83,6f477eb9c90036177a1a870f0d4d5cc8,fd5567754d768cc191c36f6aaa99c51d,89713a21b54d7e0c55542ee840df5edf,2e289caa87e3a7af21c26b12f60b0ac9,Noelaerhabdaceae
Lasker18Sc122_17,0.006743,0.002309,0.219094,0.000000,0.006297,0.001286,0.002335,0.000000,0.015258,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010784
Lasker18Sc122_19,0.006891,0.002272,0.219868,0.000000,0.005595,0.000884,0.002302,0.000000,0.014071,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014332
Lasker18Sc122_3,0.051873,0.002164,0.024401,0.000583,0.000897,0.032282,0.000601,0.024567,0.003570,0.024891,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031148
Lasker18Sc122_5,0.051078,0.001516,0.024989,0.000000,0.001002,0.040638,0.000538,0.026383,0.004132,0.023718,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023434
Lasker18Sc123_17,0.000390,0.175130,0.043393,0.003064,0.010398,0.000817,0.013554,0.000000,0.008058,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lasker18Sc154_5,0.111341,0.002494,0.011221,0.003206,0.001091,0.040542,0.001225,0.048491,0.009707,0.043904,...,0.000022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000493
Lasker18Sc158_17,0.001213,0.122211,0.003561,0.108075,0.046169,0.001819,0.081640,0.000917,0.013529,0.000000,...,0.000000,0.000016,0.000016,0.000016,0.000016,0.000000,0.000000,0.000000,0.000000,0.005269
Lasker18Sc158_19,0.000904,0.131803,0.002878,0.114350,0.042046,0.001439,0.070698,0.000886,0.013579,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006407
Lasker18Sc158_3,0.105980,0.008528,0.044897,0.004708,0.005774,0.073538,0.005650,0.029920,0.040989,0.012703,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000018,0.000000,0.000000,0.000000,0.006211


#### Save the merged table as a csv

In [113]:
df.to_csv("RREAS2018_16S_ASVs_with_Noelaerhab.csv")